In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import torch
from models.vt_resnet import VTResNet
from models.resnet import BasicBlock

In [3]:
N = 5
L = 3
C = 7
H = 224
W = 224
D = 10

x = torch.randn((N, 3,H, W))
# print(x)

In [4]:
vtresnet = VTResNet(
    resnet_block=BasicBlock,
    layers=[2,2,2,2],
    tokens=L,
    token_channels=D,
    num_classes=3
)
# print(vtresnet)

In [ ]:
import torch